# import

In [ ]:
import torch
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
import itertools
from sklearn import metrics
from matplotlib import pyplot as plt

In [ ]:
torch.cuda.empty_cache()

In [ ]:
device=torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

In [ ]:
device=torch.device("cpu")

# data

## data1

In [ ]:
def load_data():  # lnc 240 dis 405 mi 495
    ld=np.loadtxt("./data/lnc_dis_association.txt",dtype=int)
    dd=np.loadtxt("./data/dis_sim_matrix_process.txt",dtype=float)
    md=np.loadtxt("./data/mi_dis.txt",dtype=int)
    lm=np.loadtxt("./data/yuguoxian_lnc_mi.txt",dtype=int)
    return torch.tensor(ld),torch.tensor(dd),torch.tensor(md),torch.tensor(lm)

In [ ]:
def calculate_sim(ld,dd):
    s1=ld.shape[0]
    ll=torch.eye(s1)
    m2=dd*ld[:,None,:]
    m1=ld[:,:,None]
    for x,y in itertools.permutations(torch.linspace(0,s1-1,s1,dtype=torch.long),2):
        x,y=x.item(),y.item()
        m=m1[x,:,:]*m2[y,:,:]
        if ld[x].sum()+ld[y].sum()==0:
            ll[x,y]=0
        else:
            ll[x,y]=(m.max(dim=0,keepdim=True)[0].sum()+m.max(dim=1,keepdim=True)[0].sum())/(ld[x].sum()+ld[y].sum())
    return ll

In [ ]:
def split_dataset(al,dd,n):#5 cross
    rand_index=torch.randperm(al.sum())
    ps=torch.argwhere(al==1).index_select(0,rand_index)
    ns=torch.argwhere(al==0)
    ns=ns.index_select(0,torch.randperm(ns.shape[0]))
    sf=int(ps.shape[0]/n)
    tri,tei,lda,ll=[],[],[],[]
    for i in range(n):
        ptrn=torch.cat([ps[:(i*sf),:],ps[((i+1)*sf):(n*sf),:]],dim=0).T
        ntrn=torch.cat([ns[:(i*sf),:],ns[((i+1)*sf):(n*sf),:]],dim=0).T
        trn=torch.cat([ptrn,ntrn],dim=1)
        pten=torch.cat([ps[(i*sf):((i+1)*sf),:],ps[(n*sf):,:]],dim=0).T
        nten=torch.cat([ns[(i*sf):((i+1)*sf),:],ns[(n*sf):,:]],dim=0).T
        ten=torch.cat([pten,nten],dim=1)
        tri.append(trn)
        tei.append(ten)
        ldt=al.clone()
        ldt[pten[0,:],pten[1,:]]=0
        lda.append(ldt)
        ll.append(calculate_sim(ldt,dd))  
    return tri,tei,lda,ll

In [ ]:
n=5
ld,dd,md,lm=load_data()
tri,tei,lda,ll=split_dataset(ld,dd,n)
mm=calculate_sim(md,dd)
datasave=(n,ld,dd,md,lm,tri,tei,lda,ll,mm)
torch.save(datasave,'tempdata2.pth')

## data2

In [ ]:
n,ld,dd,md,lm,tri,tei,lda,ll,mm=torch.load('./tempdata2.pth')
def chm(m1,m2,th1=0,th2=0,th3=0):
    #th is threshold
    poi=[]
    m1idx=torch.argwhere(m1)
    for i in range(m1idx.shape[0]):
        p1,p2=m1idx[i]
        m2idx=torch.argwhere(m2[p2])
        for j in range(m2idx.shape[0]):
            poi.append([p1+th1,p2+th2,m2idx[j][0]+th3])
    return poi
def cfm(ll,ld,dd,md,lm,mm):
    #lld ldd lmd
    #  e1 e2 e3 ..
    #l
    #d
    #m
    #llidx=torch.argwhere(ll)  
    #ldidx=torch.argwhere(ld)
    #ddidx=torch.argwhere(dd)
    #mdidx=torch.argwhere(md)
    #lmidx=torch.argwhere(lm)
    
    r1=torch.cat([ll,ld,lm],dim=1)
    r2=torch.cat([ld.T,dd,md.T],dim=1)
    r3=torch.cat([lm.T,md,mm],dim=1)
    fea=torch.cat([r1,r2,r3],dim=0)
    
    deg=torch.diag((torch.sum(fea,dim=1))**(-1/2)).double()
    #G1=deg@(fea+torch.diag(torch.ones(fea.shape[0])))@deg
    G1=deg@fea@deg

    '''计算超图G2包含元路径信息
    #这里容易使得超图过大，将大矩阵分块计算
    pois1=chm(ll,ld,0,0,240)+chm(ld,dd,0,240,240)+chm(lm,md,0,645,240)
    pois=pois1
    # pois=[]
    #for l1 in pois1:   删除冗余。不过代价太大
    #    if l1 not in pois:
    #        pois.append(l1)
    H=[]
    Dvs=[]
    Des=[]
    Hsize=1000
    Hl=len(pois)//Hsize
    for i in range(Hl+1):
        if i==Hl:
            Ht=torch.zeros(1140,len(pois)-Hl*Hsize)
        else:
            Ht=torch.zeros(1140,Hsize)
        for j in range(Ht.shape[1]):
            Ht[pois[j+i*Hsize],j]=1
        Dvs.append(torch.sum(Ht,dim=1,keepdim=True))
        Des.append(torch.sum(Ht,dim=0,keepdim=True))
        H.append(Ht)
    Dv=torch.zeros(1140,1)
    for i in range(len(Dvs)):
        Dv+=Dvs[i]
    Dv=Dv**(-1/2)
    Dv[torch.where(Dv==torch.inf)]=0
    for i in range(len(Des)):
        Des[i]=Des[i]**(-1)
    for i in range(len(H)):
        H[i]=Dv*H[i]*Des[i]@H[i].T*Dv
    G2=torch.zeros(1140,1140)
    for i in range(len(H)):
        G2+=H[i]
    '''
    return fea,G1#,G2

In [ ]:
feas=[]
G1s=[]
#G2s=[]
for i in range(n):
    fea,G1=cfm(ll[i],lda[i],dd,md,lm,mm)
    feas.append(fea)
    G1s.append(G1)
    #G2s.append(G2)
torch.save((feas,G1s),'./tempdata3.pth')

# model

## train

In [ ]:
learn_rate=0.0003
epoch=80
batch=32
n,ld,dd,md,lm,tri,tei,lda,ll,mm=torch.load('./parasave/tempdata2.pth')
feas,G1s=torch.load('./parasave/tempdata3.pth')

In [ ]:
class MyDataset(Dataset):#shuffle and batch from Dataloader
    def __init__(self,tri,ld):
        self.tri=tri
        self.ld=ld
    def __getitem__(self,idx):
        x,y=self.tri[:,idx]
        label=self.ld[x][y]
        return x,y,label
    def __len__(self):
        return self.tri.shape[1]

In [ ]:
class HGNN_conv(nn.Module):
    '''
    @article{feng2018hypergraph,
    title={Hypergraph Neural Networks},
    author={Feng, Yifan and You, Haoxuan and Zhang, Zizhao and Ji, Rongrong and Gao, Yue},
    journal={AAAI 2019},
    year={2018}
    }
    '''
    def __init__(self, in_ft, out_ft, bias=True):
        super(HGNN_conv, self).__init__()

        self.weight = nn.Parameter(torch.Tensor(in_ft, out_ft))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_ft))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))  
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x: torch.Tensor, G: torch.Tensor):
        x = x.matmul(self.weight)     
        if self.bias is not None:
            x = x + self.bias
        x = G.matmul(x)          
        return x
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.ne=nn.Parameter(torch.rand(500,300))
        self.w1=nn.Parameter(torch.rand(1140,300))
        #self.w2=nn.Parameter(torch.randn(1440,300))
        self.w2=nn.Linear(1440,300)
        self.h3=HGNN_conv(1140,600)
        self.h4=HGNN_conv(600,300)
        self.h1=HGNN_conv(1140,600)
        self.h2=HGNN_conv(600,300)        
        self.hf1=nn.Linear(1440*2,50)

        self.c11=nn.Conv2d(1,32,kernel_size=(2,7),stride=1,padding=0)
        self.s2=nn.MaxPool2d(kernel_size=(1,7))
        self.c31=nn.Conv2d(32,64,kernel_size=(1,7),stride=1,padding=0)
        self.c32=nn.Conv2d(32,64,kernel_size=(1,7),stride=1,padding=0)
        self.s4=nn.MaxPool2d(kernel_size=(1,7))
        self.cf1=nn.Linear(22*64,200)#1140->70

        self.f3=nn.Linear(250,2)
        self.cd1=nn.Dropout(0.5)
        self.cd2=nn.Dropout2d(0.5)
        
        self.leakrelu=nn.LeakyReLU()
        self.tanh=nn.Tanh()
    def forward(self,x1,x2,fea,G1):
        
        x3=x2+240
        x=torch.cat([fea[x1][:,None,None,:],fea[x3][:,None,None,:]],dim=2)
        
        hyperH=self.leakrelu(G1@self.w1@self.ne.T)
        self.H=hyperH
        hyperH=(hyperH-hyperH.min())/(hyperH.max()-hyperH.min())
        
        dv=(torch.sum(hyperH,dim=1)**(-1/2))[:,None]
        de=(torch.sum(hyperH,dim=0)**(-1))[None,:]
        dv[torch.where(dv==torch.inf)]=0
        de[torch.where(de==torch.inf)]=0
        hyperH=dv*hyperH*de@hyperH.T*dv

        fea1=self.leakrelu(self.h1(fea,G1))
        fea2=self.leakrelu(self.h3(fea,hyperH))
        fea3=self.leakrelu(self.h2(fea1,G1))
        fea4=self.leakrelu(self.h4(fea2+fea1,hyperH))
        fea3=torch.cat([fea,fea3+fea4],dim=1)

        self.oe=self.w2(self.H.T@fea3)
        
        fea3=torch.cat([fea3[x1],fea3[x3]],dim=1)
        fea3=self.leakrelu(self.hf1(fea3))
        
        x=self.s2(self.leakrelu(self.c11(x)))
        att=self.tanh(self.c32(x))
        x=self.s4(self.leakrelu(self.c31(x)*att+self.c31(x)))
        x=self.cd2(x)
        x=x.reshape(x.shape[0],-1)
        x=self.leakrelu(self.cf1(x))
        
        x=torch.cat([x,fea3],dim=1)
        
        x=self.cd1(x)
        x=self.f3(x)

        return x
    
    def loss(self):
        p=torch.softmax(self.oe/1e+7,dim=-1)
        q=torch.log_softmax(self.ne,dim=-1)
        return nn.KLDivLoss(reduction='batchmean')(q,p)
#net=Net()
#fea=torch.randn(1140,1140)
#G=torch.randn(1140,1140)
#x1=torch.linspace(0,31,32).long()
#x2=x1
#net(x1,x2,fea,G).shape
#net.loss()

In [ ]:
def train(model,train_set,test_set,fea,G1,tei,epoch,learn_rate,cros):
    optimizer=torch.optim.Adam(model.parameters(),learn_rate,weight_decay=0.001)
    cost=nn.CrossEntropyLoss()
    model.train()
    fea,G1=fea.float().to(device),G1.float().to(device)#,G2.float().to(device)
    Amax=[0,0]
    for i in range(epoch):
        for x1,x2,y in train_set:
            x1,x2,y=Variable(x1.long()).to(device),Variable(x2.long()).to(device),Variable(y.long()).to(device)
            out=model(x1,x2,fea,G1)
            loss=cost(out,y)#+model.loss()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if (i+1)%10 == 0 and i+1>=50:
            tacc(model,test_set,fea,G1,tei,cros,Amax)
        #if i+1==epoch:
            #tacc(model,test_set,fea,G1,0,cros)
        torch.cuda.empty_cache()
'''       
def tacc(model,tset,fea,G1,string,cros):
    correct=0
    total=0
    st={1:'train_acc',0:'test_acc'}
    predall,yall=torch.tensor([]),torch.tensor([])
    model.eval()
    for x1,x2,y in tset:
        x1,x2,y=Variable(x1.long()).to(device),Variable(x2.long()).to(device),Variable(y.long()).to(device)
        pred=model(x1,x2,fea,G1).data
        predall=torch.cat([predall,torch.as_tensor(pred,device='cpu')],dim=0)
        yall=torch.cat([yall,torch.as_tensor(y,device='cpu')])
        a=torch.max(pred,1)[1]
        total+=y.size(0)
        correct+=(a==y).sum()
    print(st[string]+str((correct/total).item()))
    torch.save((predall,yall),'PandY%d' % cros)
'''
def calculate_TPR_FPR(RD, f, B):
    old_id = np.argsort(-RD)
    min_f = int(min(f))
    max_f = int(max(f))
    TP_FN = np.zeros((RD.shape[0], 1), dtype=np.float64)
    FP_TN = np.zeros((RD.shape[0], 1), dtype=np.float64)
    TP = np.zeros((RD.shape[0], max_f), dtype=np.float64)
    TP2 = np.zeros((RD.shape[0], min_f), dtype=np.float64)
    FP = np.zeros((RD.shape[0], max_f), dtype=np.float64)
    FP2 = np.zeros((RD.shape[0], min_f), dtype=np.float64)
    P = np.zeros((RD.shape[0], max_f), dtype=np.float64)
    P2 = np.zeros((RD.shape[0], min_f), dtype=np.float64)
    for i in range(RD.shape[0]):
        TP_FN[i] = sum(B[i] == 1)
        FP_TN[i] = sum(B[i] == 0)
    for i in range(RD.shape[0]):
        for j in range(int(f[i])):
            if j == 0:
                if B[i][old_id[i][j]] == 1:
                    FP[i][j] = 0
                    TP[i][j] = 1
                    P[i][j] = TP[i][j] / (j + 1)
                else:
                    TP[i][j] = 0
                    FP[i][j] = 1
                    P[i][j] = TP[i][j] / (j + 1)
            else:
                if B[i][old_id[i][j]] == 1:
                    FP[i][j] = FP[i][j - 1]
                    TP[i][j] = TP[i][j - 1] + 1
                    P[i][j] = TP[i][j] / (j + 1)
                else:
                    TP[i][j] = TP[i][j - 1]
                    FP[i][j] = FP[i][j - 1] + 1
                    P[i][j] = TP[i][j] / (j + 1)
    ki = 0
    for i in range(RD.shape[0]):
        if TP_FN[i] == 0:
            TP[i] = 0
            FP[i] = 0
            ki = ki + 1
        else:
            TP[i] = TP[i] / TP_FN[i]
            FP[i] = FP[i] / FP_TN[i]
    for i in range(RD.shape[0]):
        kk = f[i] / min_f
        for j in range(min_f):
            TP2[i][j] = TP[i][int(np.round_(((j + 1) * kk))) - 1]
            FP2[i][j] = FP[i][int(np.round_(((j + 1) * kk))) - 1]
            P2[i][j] = P[i][int(np.round_(((j + 1) * kk))) - 1]
    TPR = TP2.sum(0) / (TP.shape[0] - ki)
    FPR = FP2.sum(0) / (FP.shape[0] - ki)
    Pr = P2.sum(0) / (P.shape[0] - ki)
    return TPR, FPR, Pr
def tacc(model,tset,fea,G1,tei,cros,Amax):
    predall,yall=torch.tensor([]),torch.tensor([])
    model.eval()
    for x1,x2,y in tset:
        x1,x2,y=Variable(x1.long()).to(device),Variable(x2.long()).to(device),Variable(y.long()).to(device)
        pred=model(x1,x2,fea,G1).data
        predall=torch.cat([predall,torch.as_tensor(pred,device='cpu')],dim=0)
        yall=torch.cat([yall,torch.as_tensor(y,device='cpu')])
    #torch.save((predall,yall),'./prady/PandY%d_%d' % (cros,epoch+1))
    pred=torch.softmax(predall,dim=1)[:,1]
    trh=torch.zeros(ld.shape[0],ld.shape[1])-1
    tlh=torch.zeros(ld.shape[0],ld.shape[1])-1
    trh[tei[0],tei[1]]=pred
    tlh[tei[0],tei[1]]=yall
    R=trh.numpy()
    label=tlh.numpy()
    f = np.zeros(shape=(R.shape[0], 1))
    for i in range(R.shape[0]):
        f[i] = np.sum(R[i] > -1)
    if min(f)>0:
        TPR,FPR,P=calculate_TPR_FPR(R,f,label)
        AUC=metrics.auc(FPR, TPR)
        AUPR=metrics.auc(TPR, P) + (TPR[0] * P[0])
        print("AUC:%.4f_AUPR:%.4f"%(AUC,AUPR))
        if AUPR>Amax[1]:
            Amax[0]=AUC
            Amax[1]=AUPR
            print("save")
            torch.save((predall,yall),"PandY%d"%cros)


In [ ]:
for i in range(n):
    net=Net().to(device)
    trset=DataLoader(MyDataset(tri[i],ld),batch,shuffle=True)
    teset=DataLoader(MyDataset(tei[i],ld),batch,shuffle=False)
    train_set,test_set=[],[]
    for x1,x2,y in trset:
        train_set.append((x1,x2,y))
    for x1,x2,y in teset:
        test_set.append((x1,x2,y))
    print('cross:'+str(i+1))
    train(net,train_set,test_set,feas[i],G1s[i],tei[i],epoch,learn_rate,i)
    #torch.save(net.state_dict(),'./modelpara/model%d'%i)

## plt1

In [ ]:
def fold_5(TPR, FPR, PR):
    fold = len(TPR)
    le = []
    for i in range(fold):
        le.append(len(TPR[i]))
    min_f = min(le)
    F_TPR = np.zeros((fold, min_f))
    F_FPR = np.zeros((fold, min_f))
    F_P = np.zeros((fold, min_f))
    for i in range(fold):
        k = len(TPR[i]) / min_f
        for j in range(min_f):
            F_TPR[i][j] = TPR[i][int(round(((j + 1) * k))) - 1]
            F_FPR[i][j] = FPR[i][int(round(((j + 1) * k))) - 1]
            F_P[i][j] = PR[i][int(round(((j + 1) * k))) - 1]
    TPR_5 = F_TPR.sum(0) / fold
    FPR_5 = F_FPR.sum(0) / fold
    PR_5 = F_P.sum(0) / fold
    return TPR_5, FPR_5, PR_5
def calculate_TPR_FPR(RD, f, B):
    old_id = np.argsort(-RD)
    min_f = int(min(f))
    max_f = int(max(f))
    TP_FN = np.zeros((RD.shape[0], 1), dtype=np.float64)
    FP_TN = np.zeros((RD.shape[0], 1), dtype=np.float64)
    TP = np.zeros((RD.shape[0], max_f), dtype=np.float64)
    TP2 = np.zeros((RD.shape[0], min_f), dtype=np.float64)
    FP = np.zeros((RD.shape[0], max_f), dtype=np.float64)
    FP2 = np.zeros((RD.shape[0], min_f), dtype=np.float64)
    P = np.zeros((RD.shape[0], max_f), dtype=np.float64)
    P2 = np.zeros((RD.shape[0], min_f), dtype=np.float64)
    for i in range(RD.shape[0]):
        TP_FN[i] = sum(B[i] == 1)
        FP_TN[i] = sum(B[i] == 0)
    for i in range(RD.shape[0]):
        for j in range(int(f[i])):
            if j == 0:
                if B[i][old_id[i][j]] == 1:
                    FP[i][j] = 0
                    TP[i][j] = 1
                    P[i][j] = TP[i][j] / (j + 1)
                else:
                    TP[i][j] = 0
                    FP[i][j] = 1
                    P[i][j] = TP[i][j] / (j + 1)
            else:
                if B[i][old_id[i][j]] == 1:
                    FP[i][j] = FP[i][j - 1]
                    TP[i][j] = TP[i][j - 1] + 1
                    P[i][j] = TP[i][j] / (j + 1)
                else:
                    TP[i][j] = TP[i][j - 1]
                    FP[i][j] = FP[i][j - 1] + 1
                    P[i][j] = TP[i][j] / (j + 1)
    ki = 0
    for i in range(RD.shape[0]):
        if TP_FN[i] == 0:
            TP[i] = 0
            FP[i] = 0
            ki = ki + 1
        else:
            TP[i] = TP[i] / TP_FN[i]
            FP[i] = FP[i] / FP_TN[i]
    for i in range(RD.shape[0]):
        kk = f[i] / min_f
        for j in range(min_f):
            TP2[i][j] = TP[i][int(np.round_(((j + 1) * kk))) - 1]
            FP2[i][j] = FP[i][int(np.round_(((j + 1) * kk))) - 1]
            P2[i][j] = P[i][int(np.round_(((j + 1) * kk))) - 1]
    TPR = TP2.sum(0) / (TP.shape[0] - ki)
    FPR = FP2.sum(0) / (FP.shape[0] - ki)
    Pr = P2.sum(0) / (P.shape[0] - ki)
    return TPR, FPR, Pr
def curve(FPR, TPR, P):
    plt.figure()
    plt.subplot(121)
    plt.xlim(0.0, 1.0)
    plt.ylim(0.0, 1.0)
    plt.title("ROC curve  (AUC = %.4f)" % (metrics.auc(FPR, TPR)))
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.plot(FPR, TPR)
    plt.subplot(122)
    plt.xlim(0.0, 1.0)
    plt.ylim(0.0, 1.0)
    plt.title("PR curve  (AUPR = %.4f)" % (metrics.auc(TPR, P) + (TPR[0] * P[0])))
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.plot(TPR, P)
    plt.show()

In [ ]:
def calculate_AUC_AUPR(Rh,labelh):
    FPRs = []
    TPRs = []
    Ps = []
    for step in range(5):
        R=Rh[step]
        label=labelh[step]
        R=R
        label=label
        f = np.zeros(shape=(R.shape[0], 1))
        for i in range(R.shape[0]):
            f[i] = np.sum(R[i] > -1)
        TPR, FPR, P = calculate_TPR_FPR(R, f, label)
        FPRs.append(FPR)
        TPRs.append(TPR)
        Ps.append(P)
    TPR_5, FPR_5, PR_5 = fold_5(TPRs, FPRs, Ps)
    #np.savetxt("./data/compare/hyper/TPR.txt",TPR_5)
    #np.savetxt("./data/compare/hyper/FPR.txt",FPR_5)
    #np.savetxt("./data/compare/hyper/P.txt",PR_5)
    curve(FPR_5, TPR_5, PR_5)

In [ ]:
n,ld,_,_,_,_,tei,_,_,_=torch.load('./parasave/tempdata2.pth')
Rh,labelh=[],[]
for i in range(n):
    pred,y=torch.load('./PandY%d'%i)
    pred=torch.softmax(pred,dim=1)[:,1]
    trh=torch.zeros(ld.shape[0],ld.shape[1])-1
    tlh=torch.zeros(ld.shape[0],ld.shape[1])-1
    trh[tei[i][0],tei[i][1]]=pred
    tlh[tei[i][0],tei[i][1]]=y
    Rh.append(trh.numpy())
    labelh.append(tlh.numpy())
calculate_AUC_AUPR(Rh,labelh)

# plt2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from  scipy.stats import ttest_rel
import torch
import pandas as pd

## ROC and AUPR

In [ ]:
hyper_TPR,hyper_FPR=np.loadtxt("./data/compare/hyper/TPR.txt"),np.loadtxt("./data/compare/hyper/FPR.txt")
hyper_P,hyper_R=np.loadtxt("./data/compare/hyper/P.txt"),np.loadtxt("./data/compare/hyper/TPR.txt")
GAIRD_TPR,GAIRD_FPR=np.loadtxt("./data/compare/GAIRD/TPR.txt"),np.loadtxt("./data/compare/GAIRD/FPR.txt")
GAIRD_P,GAIRD_R=np.loadtxt("./data/compare/GAIRD/P.txt"),np.loadtxt("./data/compare/GAIRD/TPR.txt")
GSMV_TPR,GSMV_FPR=np.loadtxt("./data/compare/GSMV/TPR.txt"),np.loadtxt("./data/compare/GSMV/FPR.txt")
GSMV_P,GSMV_R=np.loadtxt("./data/compare/GSMV/P.txt"),np.loadtxt("./data/compare/GSMV/TPR.txt")
MGLDA_TPR,MGLDA_FPR=np.loadtxt("./data/compare/MGLDA_Result/TPR.txt"),np.loadtxt("./data/compare/MGLDA_Result/FPR.txt")
MGLDA_P,MGLDA_R=np.loadtxt("./data/compare/MGLDA_Result/P.txt"),np.loadtxt("./data/compare/MGLDA_Result/TPR.txt")
GTAN_TPR,GTAN_FPR=np.loadtxt("./data/compare/GTAN_Result/TPR.txt"),np.loadtxt("./data/compare/GTAN_Result/FPR.txt")
GTAN_P,GTAN_R=np.loadtxt("./data/compare/GTAN_Result/P.txt"),np.loadtxt("./data/compare/GTAN_Result/R.txt")
VADLP_TPR,VADLP_FPR=np.loadtxt("./data/compare/VADLP_Result/VADLP_TPR.txt"),np.loadtxt("./data/compare/VADLP_Result/VADLP_FPR.txt")
VADLP_P,VADLP_R=np.loadtxt("./data/compare/VADLP_Result/VADLP_P.txt"),np.loadtxt("./data/compare/VADLP_Result/VADLP_TPR.txt")
CNNLDA_TPR,CNNLDA_FPR=np.loadtxt("./data/compare/Four_methods_Result/CNNLDA_TPR.txt"),np.loadtxt("./data/compare/Four_methods_Result/CNNLDA_FPR.txt")
CNNLDA_P,CNNLDA_R=np.loadtxt("./data/compare/Four_methods_Result/CNNLDA_P.txt"),np.loadtxt("./data/compare/Four_methods_Result/CNNLDA_TPR.txt")

In [ ]:
fig=plt.figure(figsize=(10,5))
ax=plt.subplot(121)
a1, = plt.plot(hyper_FPR,hyper_TPR,'darkviolet')
a2, = plt.plot(GAIRD_FPR,GAIRD_TPR,'y')
a3, = plt.plot(GSMV_FPR,GSMV_TPR,'c')
a4, = plt.plot(MGLDA_FPR,MGLDA_TPR,'pink')
a5, = plt.plot(GTAN_FPR,GTAN_TPR,'g')
a6, = plt.plot(VADLP_FPR,VADLP_TPR,'b')
a7, = plt.plot(CNNLDA_FPR,CNNLDA_TPR,'r')
plt.legend([a1,a2,a3,a4,a5,a6,a7],["AGLDA(%.3f)"%(auc(hyper_FPR,hyper_TPR)),
                                "GAIRD(%.3f)"%(auc(GAIRD_FPR,GAIRD_TPR)),
                                "GSMV(%.3f)"%(auc(GSMV_FPR,GSMV_TPR)),
                                "MGLDA(%.3f)"%(auc(MGLDA_FPR,MGLDA_TPR)),
                                "GTAN(%.3f)"%(auc(GTAN_FPR,GTAN_TPR)),
                                "VADLP(%.3f)"%(auc(VADLP_FPR,VADLP_TPR)),
                                "CNNLDA(%.3f)"%(auc(CNNLDA_FPR,CNNLDA_TPR))],loc="best",fontsize=6)
ax.set_title("(A)ROC curves",fontsize=10,)
plt.xlabel("FPR")
plt.ylabel("TPR")
#fig.savefig("./data/compare/hyper/AUC.svg",format="svg")
ax1=plt.subplot(122)
a1, = plt.plot(hyper_R,hyper_P,'darkviolet')
a2, = plt.plot(GAIRD_R,GAIRD_P,'y')
a3, = plt.plot(GSMV_R,GSMV_P,'c')
a4, = plt.plot(MGLDA_R,MGLDA_P,'pink')
a5, = plt.plot(GTAN_R,GTAN_P,'g')
a6, = plt.plot(VADLP_R,VADLP_P,'b')
a7, = plt.plot(CNNLDA_R,CNNLDA_P,'r')
plt.legend([a1,a2,a3,a4,a5,a6,a7],["AGLDA(%.3f)"%(auc(hyper_R,hyper_P)+hyper_R[0]*hyper_P[0]),
                                   "GAIRD(%.3f)"%(auc(GAIRD_R,GAIRD_P)+GAIRD_R[0]*GAIRD_P[0]),
                                "GSMV(%.3f)"%(auc(GSMV_R,GSMV_P)+GSMV_R[0]*GSMV_P[0]),
                                "MGLDA(%.3f)"%(auc(MGLDA_R,MGLDA_P)+MGLDA_R[0]*MGLDA_P[0]),
                                "GTAN(%.3f)"%(auc(GTAN_R,GTAN_P)+GTAN_R[0]*GTAN_P[0]),
                                "VADLP(%.3f)"%(auc(VADLP_R,VADLP_P)+VADLP_R[0]*VADLP_P[0]),
                                "CNNLDA(%.3f)"%(auc(CNNLDA_R,CNNLDA_P)+CNNLDA_R[0]*CNNLDA_P[0])],loc="best",fontsize=6)
ax1.set_title("(B)PR curves",fontsize=10,)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.plot(linewidth=15)
plt.show()
fig.savefig("./data/compare/hyper/AUCandAUPR.svg",format="svg")

## top k

In [ ]:
hyper_Recall=[0.94173030,0.96624893,0.97417315,0.98944983,0.99286410,0.99367440,0.99626938,1]
GAIRD_Rcall=[0.94024780,0.96510977,0.97338228,0.98642203,0.99186745,0.99565807,0.99815678,1]
GSMV_Recall=[0.92372811,0.94158490,0.96736856,0.98186581,0.99045652,0.99177033,0.99302148,1]
MGLDA_Recall=[0.91193719,0.93337950,0.96638335,0.97307928,0.98364267,0.99065807,0.99167848,1]
GTAN_Recall  =  [0.89719380, 0.92813318, 0.94834836, 0.96815835, 0.97670324, 0.98511180, 0.98977049, 1.]
VADLP_Recall = [0.85157542, 0.91083547, 0.92468019, 0.94498560, 0.97638439, 0.97972933, 0.98801061, 1.]
CNNLDA_Recall = [0.74630222, 0.8952459, 0.91343092, 0.93538218, 0.97359227, 0.9738827, 0.97458565, 1.]
fig = plt.figure(figsize=(7,5),dpi=100)#
ax2 = fig.add_subplot(1,1,1)
size = 8
x = np.arange(size)
total_width, n = 0.8, 8 
width = total_width / n
x = x - (total_width - width) / 2
ax2.set_xticks(x)
plt.bar(x - 0.3, hyper_Recall, width=width, label='AGLDA', color='darkviolet')
plt.bar(x - 0.3 + 1*width, GAIRD_Rcall, width=width, label='GAIRD', color='g') 
plt.bar(x - 0.3 + 2*width, GSMV_Recall, width=width, label='GSMV', color='b') 
plt.bar(x - 0.3 + 3 *width, MGLDA_Recall, width=width, label='MGLDA', color='r') 
plt.bar(x - 0.3 + 4 * width, GTAN_Recall, width=width, label='GTAN', color='y') 
plt.bar(x - 0.3 + 5 * width, VADLP_Recall, width=width, label="VADLP", color='c')
plt.bar(x - 0.3 + 6 * width, CNNLDA_Recall, width=width, label='CNNLDA', color='pink')
ax2.set_xticklabels(['Top30', 'Top60', 'Top90', 'Top120', 'Top150', 'Top180', 'Top210', 'Top240', ])
box = ax2.get_position()
ax2.set_position([box.x0, box.y0, box.width, box.height * 1.1])
ax2.legend(loc='upper left', bbox_to_anchor=(0.06, 1.14),ncol=4)
#plt.legend(loc='NorthOutside')
plt.ylabel("Recall",fontsize='large')
plt.show()
fig.savefig("./data/compare/hyper/top_k.svg",format="svg",bbox_inches='tight')

## p-value

In [ ]:
n,ld,dd,md,lm,tri,tei,lda,ll,mm=torch.load('./parasave/tempdata2.pth')
numm=torch.zeros(240,405)
predld=torch.zeros(240,405)
yld=torch.zeros(240,405)
for i in range(n):
    pred,y=torch.load('./parasave/PandY%d_9877_6841'%i)
    pred=torch.softmax(pred,dim=1)[:,1]
    trh=torch.zeros(ld.shape[0],ld.shape[1])
    tlh=torch.zeros(ld.shape[0],ld.shape[1])
    numt=torch.zeros(ld.shape[0],ld.shape[1])
    trh[tei[i][0],tei[i][1]]=pred
    tlh[tei[i][0],tei[i][1]]=y
    numt[tei[i][0],tei[i][1]]=1
    numm+=numt
    predld+=trh
    yld+=tlh
predld[35,225],predld[216,346]=0,0.9522#去除nan
predld=(predld/numm).numpy()
yld=(yld/numm).numpy()

In [ ]:
def calculate_10_TPR_FPR_PR(F, index, dis_mi):
    SS = np.argsort(-F)
    v, w = [], []
    for i in range(len(index)):
        Precision, TPR, FPR = [], [], []
        precisim_num, TPR_NUM, FPR_NUM = 0.0, 0.0, 0.0
        b = sum(F[index[i]] > 0)
        # print(b)
        c = sum(dis_mi[index[i]] == 1)
        # print(c)
        d = b - c
        J = 0
        for j in range(b):
            if dis_mi[index[i], SS[index[i], j]] == 1:
                J += 1
                TPR_NUM += 1 / c
                TPR.append(TPR_NUM)
                FPR.append(FPR_NUM)
                Precision.append(J / (j + 1))
            else:
                FPR_NUM += 1 / d
                TPR.append(TPR_NUM)
                FPR.append(FPR_NUM)
                Precision.append(J / (j + 1))
        v.append(auc(FPR, TPR))
        w.append(auc(TPR, Precision))
    return v,w
dis_10=[9,61,62,69,113,140,178,187,211,233]
#n,ld,_,_,_,_,tei,_,_,_=torch.load('./parasave/tempdata2.pth')
#dis_10_ROCs,dis_10_PRs=[0]*10,[0]*10
#for i in range(n):
#    pred,y=torch.load('./parasave/PandY%d_9877_6841'%i)
#    pred=torch.softmax(pred,dim=1)[:,1]
#    trh=torch.zeros(ld.shape[0],ld.shape[1])
#    tlh=torch.zeros(ld.shape[0],ld.shape[1])
#    trh[tei[i][0],tei[i][1]]=pred
#    tlh[tei[i][0],tei[i][1]]=y
#    lnc_dis1=tlh.numpy()
#    Score_lnc_dis=trh.numpy()
#    dis_10_ROC,dis_10_PR=calculate_10_TPR_FPR_PR(lnc_dis1.T,dis_10,Score_lnc_dis.T)
#    dis_10_ROCs=[dis_10_ROCs[i]+dis_10_ROC[i] for i in range(10)]
#    dis_10_PRs=[dis_10_PRs[i]+dis_10_PR[i] for i in range(10)]
#hyper_auc=[dis_10_ROCs[i]/n for i in range(10)]
#hyper_aupr=[dis_10_PRs[i]/n for i in range(10)]
dis_10_ROC,dis_10_PR=calculate_10_TPR_FPR_PR(yld.T,dis_10,predld.T)
#print(dis_10_ROC)
#print(dis_10_PR)
hyper_auc=dis_10_ROC
hyper_aupr=dis_10_PR
GAIRD_auc=[1,0.922,1,0.981,0.914,1,0.922,0.975,1,1]
GAIRD_aupr=[0.438,0.091,0.531,1,0.425,1,0.781,0.626,0.361,0.894]
GSMV_auc=[1,1,0.829,0.830,0.999,0.900,1,0.940,1,0.870]
GSMV_aupr=[1,1,0.860,0.860,0.974,0.974,0.954,0.754,0.766,0.980]
MGLDA_auc=[1,1,0.997,0.986,0.999,0.998,0.999,1,1,0.997]
MGLDA_aupr=[1,1,0.850,0.840,0.974,0.964,0.944,0.744,0.736,0.970]
GATN_auc=[0.994,0.987,0.996,1.000,0.984,0.990,0.992,1.000,0.974,0.995]
GATN_aupr=[0.924,0.553,0.727,0.500,0.642,0.652,0.593,0.667,0.758,0.793]
VADLP_auc=[0.951,0.934,0.963,0.984,0.975,0.978,0.985,0.981,0.961,0.893]
VADLP_aupr=[0.372,0.203,0.624,0.238,0.461,0.825,0.837,0.628,0.37,0.205]
CNNLDA_auc=[0.928,0.922,0.886,0.981,0.914,0.953,0.86,0.975,0.932,0.932]
CNNLDA_aupr=[0.338,0.091,0.531,0.709,0.425,0.83,0.781,0.526,0.361,0.394]
auc1 = ttest_rel(hyper_auc, GAIRD_auc)
aupr1 = ttest_rel(hyper_aupr, GAIRD_aupr)
auc2 = ttest_rel(hyper_auc, GSMV_auc)
aupr2 = ttest_rel(hyper_aupr, GSMV_aupr)
auc3 = ttest_rel(hyper_auc, MGLDA_auc)
aupr3 = ttest_rel(hyper_aupr, MGLDA_aupr)
auc4 = ttest_rel(hyper_auc, GATN_auc)
aupr4 = ttest_rel(hyper_aupr, GATN_aupr)
auc5 = ttest_rel(hyper_auc, VADLP_auc)
aupr5 = ttest_rel(hyper_aupr, VADLP_aupr)
auc6 = ttest_rel(hyper_auc, CNNLDA_auc)
aupr6 = ttest_rel(hyper_aupr, CNNLDA_aupr)
auc1,aupr1,auc2,aupr2,auc3,aupr3,auc4,aupr4,auc5,aupr5,auc6,aupr6

## ablation test

### dismiss biological characteristic topological learning

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.c11=nn.Conv2d(1,32,kernel_size=(2,7),stride=1,padding=0)
        self.s2=nn.MaxPool2d(kernel_size=(1,7))
        self.c31=nn.Conv2d(32,64,kernel_size=(1,7),stride=1,padding=0)
        self.c32=nn.Conv2d(32,64,kernel_size=(1,7),stride=1,padding=0)
        self.s4=nn.MaxPool2d(kernel_size=(1,7))
        self.cf1=nn.Linear(22*64,200)#1140->70
        self.f3=nn.Linear(200,2)
        self.cd1=nn.Dropout(0.5)
        self.cd2=nn.Dropout2d(0.5)
        self.leakrelu=nn.LeakyReLU()
        self.tanh=nn.Tanh()
    def forward(self,x1,x2,fea,G1):
        x3=x2+240
        x=torch.cat([fea[x1][:,None,None,:],fea[x3][:,None,None,:]],dim=2)
        x=self.s2(self.leakrelu(self.c11(x)))
        att=self.tanh(self.c32(x))
        x=self.s4(self.leakrelu(self.c31(x)*att+self.c31(x)))
        x=self.cd2(x)
        x=x.reshape(x.shape[0],-1)
        x=self.leakrelu(self.cf1(x))
        x=self.cd1(x)
        x=self.f3(x)
        return x
#net=Net()
#fea=torch.randn(1140,1140)
#G=torch.randn(1140,1140)
#x1=torch.linspace(0,31,32).long()
#x2=x1
#net(x1,x2,fea,G).shape
##0.9869   0.6673

### dismiss pairwise node attribute learning

In [ ]:
class HGNN_conv(nn.Module):
    '''
    @article{feng2018hypergraph,
    title={Hypergraph Neural Networks},
    author={Feng, Yifan and You, Haoxuan and Zhang, Zizhao and Ji, Rongrong and Gao, Yue},
    journal={AAAI 2019},
    year={2018}
    }
    '''
    def __init__(self, in_ft, out_ft, bias=True):
        super(HGNN_conv, self).__init__()

        self.weight = nn.Parameter(torch.Tensor(in_ft, out_ft))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_ft))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))  
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x: torch.Tensor, G: torch.Tensor):
        x = x.matmul(self.weight)     
        if self.bias is not None:
            x = x + self.bias
        x = G.matmul(x)          
        return x
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.ne=nn.Parameter(torch.rand(500,300))
        self.w1=nn.Parameter(torch.rand(1140,300))
        #self.w2=nn.Parameter(torch.randn(1440,300))
        self.w2=nn.Linear(1440,300)
        self.h3=HGNN_conv(1140,600)
        self.h4=HGNN_conv(600,300)
        self.h1=HGNN_conv(1140,600)
        self.h2=HGNN_conv(600,300)        
        self.hf1=nn.Linear(1440*2,50)
        self.f3=nn.Linear(50,2)
        self.cd1=nn.Dropout(0.5)
        self.cd2=nn.Dropout2d(0.5)
        
        self.leakrelu=nn.LeakyReLU()
        self.tanh=nn.Tanh()
    def forward(self,x1,x2,fea,G1):
        
        x3=x2+240

        hyperH=self.leakrelu(G1@self.w1@self.ne.T)
        self.H=hyperH
        hyperH=(hyperH-hyperH.min())/(hyperH.max()-hyperH.min())
        
        dv=(torch.sum(hyperH,dim=1)**(-1/2))[:,None]
        de=(torch.sum(hyperH,dim=0)**(-1))[None,:]
        dv[torch.where(dv==torch.inf)]=0
        de[torch.where(de==torch.inf)]=0
        hyperH=dv*hyperH*de@hyperH.T*dv

        fea1=self.leakrelu(self.h1(fea,G1))
        fea2=self.leakrelu(self.h3(fea,hyperH))
        fea3=self.leakrelu(self.h2(fea1,G1))
        fea4=self.leakrelu(self.h4(fea2+fea1,hyperH))
        fea3=torch.cat([fea,fea3+fea4],dim=1)

        self.oe=self.w2(self.H.T@fea3)
        
        fea3=torch.cat([fea3[x1],fea3[x3]],dim=1)
        fea3=self.leakrelu(self.hf1(fea3))
        x=self.cd1(fea3)
        x=self.f3(x)
        return x
    
    def loss(self):
        p=torch.softmax(self.oe/1e+7,dim=-1)
        q=torch.log_softmax(self.ne,dim=-1)
        return nn.KLDivLoss(reduction='batchmean')(q,p)
#net=Net()
#fea=torch.randn(1140,1140)
#G=torch.randn(1140,1140)
#x1=torch.linspace(0,31,32).long()
#x2=x1
#net(x1,x2,fea,G).shape

### dismiss self-learning hypergraph

In [ ]:
class HGNN_conv(nn.Module):
    '''
    @article{feng2018hypergraph,
    title={Hypergraph Neural Networks},
    author={Feng, Yifan and You, Haoxuan and Zhang, Zizhao and Ji, Rongrong and Gao, Yue},
    journal={AAAI 2019},
    year={2018}
    }
    '''
    def __init__(self, in_ft, out_ft, bias=True):
        super(HGNN_conv, self).__init__()

        self.weight = nn.Parameter(torch.Tensor(in_ft, out_ft))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_ft))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))  
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x: torch.Tensor, G: torch.Tensor):
        x = x.matmul(self.weight)     
        if self.bias is not None:
            x = x + self.bias
        x = G.matmul(x)          
        return x
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()

        self.h1=HGNN_conv(1140,600)
        self.h2=HGNN_conv(600,300)        
        self.hf1=nn.Linear(1440*2,50)

        self.c11=nn.Conv2d(1,32,kernel_size=(2,7),stride=1,padding=0)
        self.s2=nn.MaxPool2d(kernel_size=(1,7))
        self.c31=nn.Conv2d(32,64,kernel_size=(1,7),stride=1,padding=0)
        self.c32=nn.Conv2d(32,64,kernel_size=(1,7),stride=1,padding=0)
        self.s4=nn.MaxPool2d(kernel_size=(1,7))
        self.cf1=nn.Linear(22*64,200)#1140->70

        self.f3=nn.Linear(250,2)
        self.cd1=nn.Dropout(0.5)
        self.cd2=nn.Dropout2d(0.5)
        
        self.leakrelu=nn.LeakyReLU()
        self.tanh=nn.Tanh()
    def forward(self,x1,x2,fea,G1):
        
        x3=x2+240
        x=torch.cat([fea[x1][:,None,None,:],fea[x3][:,None,None,:]],dim=2)
        
        fea1=self.leakrelu(self.h1(fea,G1))
        fea3=self.leakrelu(self.h2(fea1,G1))
        fea3=torch.cat([fea,fea3],dim=1)
        
        fea3=torch.cat([fea3[x1],fea3[x3]],dim=1)
        fea3=self.leakrelu(self.hf1(fea3))
        
        x=self.s2(self.leakrelu(self.c11(x)))
        att=self.tanh(self.c32(x))
        x=self.s4(self.leakrelu(self.c31(x)*att+self.c31(x)))
        x=self.cd2(x)
        x=x.reshape(x.shape[0],-1)
        x=self.leakrelu(self.cf1(x))
        
        x=torch.cat([x,fea3],dim=1)
        
        x=self.cd1(x)
        x=self.f3(x)

        return x
#net=Net()
#fea=torch.randn(1140,1140)
#G=torch.randn(1140,1140)
#x1=torch.linspace(0,31,32).long()
#x2=x1
#net(x1,x2,fea,G).shape

### dismiss topological learning

In [ ]:
class HGNN_conv(nn.Module):
    '''
    @article{feng2018hypergraph,
    title={Hypergraph Neural Networks},
    author={Feng, Yifan and You, Haoxuan and Zhang, Zizhao and Ji, Rongrong and Gao, Yue},
    journal={AAAI 2019},
    year={2018}
    }
    '''
    def __init__(self, in_ft, out_ft, bias=True):
        super(HGNN_conv, self).__init__()

        self.weight = nn.Parameter(torch.Tensor(in_ft, out_ft))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_ft))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))  
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x: torch.Tensor, G: torch.Tensor):
        x = x.matmul(self.weight)     
        if self.bias is not None:
            x = x + self.bias
        x = G.matmul(x)          
        return x
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.ne=nn.Parameter(torch.rand(500,300))
        self.w1=nn.Parameter(torch.rand(1140,300))
        self.w2=nn.Linear(1440,300)
        self.h3=HGNN_conv(1140,600)
        self.h4=HGNN_conv(600,300)
        self.hf1=nn.Linear(1440*2,50)

        self.c11=nn.Conv2d(1,32,kernel_size=(2,7),stride=1,padding=0)
        self.s2=nn.MaxPool2d(kernel_size=(1,7))
        self.c31=nn.Conv2d(32,64,kernel_size=(1,7),stride=1,padding=0)
        self.c32=nn.Conv2d(32,64,kernel_size=(1,7),stride=1,padding=0)
        self.s4=nn.MaxPool2d(kernel_size=(1,7))
        self.cf1=nn.Linear(22*64,200)#1140->70

        self.f3=nn.Linear(250,2)
        self.cd1=nn.Dropout(0.5)
        self.cd2=nn.Dropout2d(0.5)
        
        self.leakrelu=nn.LeakyReLU()
        self.tanh=nn.Tanh()
    def forward(self,x1,x2,fea,G1):
        
        x3=x2+240
        x=torch.cat([fea[x1][:,None,None,:],fea[x3][:,None,None,:]],dim=2)
        
        hyperH=self.leakrelu(G1@self.w1@self.ne.T)
        self.H=hyperH
        hyperH=(hyperH-hyperH.min())/(hyperH.max()-hyperH.min())
        
        dv=(torch.sum(hyperH,dim=1)**(-1/2))[:,None]
        de=(torch.sum(hyperH,dim=0)**(-1))[None,:]
        dv[torch.where(dv==torch.inf)]=0
        de[torch.where(de==torch.inf)]=0
        hyperH=dv*hyperH*de@hyperH.T*dv

        fea2=self.leakrelu(self.h3(fea,hyperH))
        fea4=self.leakrelu(self.h4(fea2,hyperH))
        fea3=torch.cat([fea,fea4],dim=1)

        self.oe=self.w2(self.H.T@fea3)
        
        fea3=torch.cat([fea3[x1],fea3[x3]],dim=1)
        fea3=self.leakrelu(self.hf1(fea3))
        
        x=self.s2(self.leakrelu(self.c11(x)))
        att=self.tanh(self.c32(x))
        x=self.s4(self.leakrelu(self.c31(x)*att+self.c31(x)))
        x=self.cd2(x)
        x=x.reshape(x.shape[0],-1)
        x=self.leakrelu(self.cf1(x))
        
        x=torch.cat([x,fea3],dim=1)
        
        x=self.cd1(x)
        x=self.f3(x)

        return x
    
    def loss(self):
        p=torch.softmax(self.oe/1e+7,dim=-1)
        q=torch.log_softmax(self.ne,dim=-1)
        return nn.KLDivLoss(reduction='batchmean')(q,p)
#net=Net()
#fea=torch.randn(1140,1140)
#G=torch.randn(1140,1140)
#x1=torch.linspace(0,31,32).long()
#x2=x1
#net(x1,x2,fea,G).shape

### dismiss hyperedge update

In [ ]:
class HGNN_conv(nn.Module):
    '''
    @article{feng2018hypergraph,
    title={Hypergraph Neural Networks},
    author={Feng, Yifan and You, Haoxuan and Zhang, Zizhao and Ji, Rongrong and Gao, Yue},
    journal={AAAI 2019},
    year={2018}
    }
    '''
    def __init__(self, in_ft, out_ft, bias=True):
        super(HGNN_conv, self).__init__()

        self.weight = nn.Parameter(torch.Tensor(in_ft, out_ft))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_ft))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))  
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x: torch.Tensor, G: torch.Tensor):
        x = x.matmul(self.weight)     
        if self.bias is not None:
            x = x + self.bias
        x = G.matmul(x)          
        return x
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.ne=nn.Parameter(torch.rand(500,300))
        self.w1=nn.Parameter(torch.rand(1140,300))
        self.h3=HGNN_conv(1140,600)
        self.h4=HGNN_conv(600,300)
        self.h1=HGNN_conv(1140,600)
        self.h2=HGNN_conv(600,300)        
        self.hf1=nn.Linear(1440*2,50)

        self.c11=nn.Conv2d(1,32,kernel_size=(2,7),stride=1,padding=0)
        self.s2=nn.MaxPool2d(kernel_size=(1,7))
        self.c31=nn.Conv2d(32,64,kernel_size=(1,7),stride=1,padding=0)
        self.c32=nn.Conv2d(32,64,kernel_size=(1,7),stride=1,padding=0)
        self.s4=nn.MaxPool2d(kernel_size=(1,7))
        self.cf1=nn.Linear(22*64,200)#1140->70

        self.f3=nn.Linear(250,2)
        self.cd1=nn.Dropout(0.5)
        self.cd2=nn.Dropout2d(0.5)
        
        self.leakrelu=nn.LeakyReLU()
        self.tanh=nn.Tanh()
    def forward(self,x1,x2,fea,G1):
        
        x3=x2+240
        x=torch.cat([fea[x1][:,None,None,:],fea[x3][:,None,None,:]],dim=2)
        
        hyperH=self.leakrelu(G1@self.w1@self.ne.T)
        self.H=hyperH
        hyperH=(hyperH-hyperH.min())/(hyperH.max()-hyperH.min())
        
        dv=(torch.sum(hyperH,dim=1)**(-1/2))[:,None]
        de=(torch.sum(hyperH,dim=0)**(-1))[None,:]
        dv[torch.where(dv==torch.inf)]=0
        de[torch.where(de==torch.inf)]=0
        hyperH=dv*hyperH*de@hyperH.T*dv

        fea1=self.leakrelu(self.h1(fea,G1))
        fea2=self.leakrelu(self.h3(fea,hyperH))
        fea3=self.leakrelu(self.h2(fea1,G1))
        fea4=self.leakrelu(self.h4(fea2+fea1,hyperH))
        fea3=torch.cat([fea,fea3+fea4],dim=1)
  
        fea3=torch.cat([fea3[x1],fea3[x3]],dim=1)
        fea3=self.leakrelu(self.hf1(fea3))
        
        x=self.s2(self.leakrelu(self.c11(x)))
        att=self.tanh(self.c32(x))
        x=self.s4(self.leakrelu(self.c31(x)*att+self.c31(x)))
        x=self.cd2(x)
        x=x.reshape(x.shape[0],-1)
        x=self.leakrelu(self.cf1(x))
        
        x=torch.cat([x,fea3],dim=1)
        
        x=self.cd1(x)
        x=self.f3(x)

        return x
#net=Net()
#fea=torch.randn(1140,1140)
#G=torch.randn(1140,1140)
#x1=torch.linspace(0,31,32).long()
#x2=x1
#net(x1,x2,fea,G).shape

## 50 xls

In [ ]:
def load_data():  # lnc 240 dis 405 mi 495
    ld=np.loadtxt("./data/lnc_dis_association.txt",dtype=int)
    dd=np.loadtxt("./data/dis_sim_matrix_process.txt",dtype=float)
    md=np.loadtxt("./data/mi_dis.txt",dtype=int)
    lm=np.loadtxt("./data/yuguoxian_lnc_mi.txt",dtype=int)
    ll=np.loadtxt("./data/lnc_sim.txt",dtype=float)
    return torch.tensor(ld),torch.tensor(dd),torch.tensor(md),torch.tensor(lm),torch.tensor(ll)
def calculate_sim(ld,dd):
    s1=ld.shape[0]
    ll=torch.eye(s1)
    m2=dd*ld[:,None,:]
    m1=ld[:,:,None]
    for x,y in itertools.permutations(torch.linspace(0,s1-1,s1,dtype=torch.long),2):
        x,y=x.item(),y.item()
        m=m1[x,:,:]*m2[y,:,:]
        if ld[x].sum()+ld[y].sum()==0:
            ll[x,y]=0
        else:
            ll[x,y]=(m.max(dim=0,keepdim=True)[0].sum()+m.max(dim=1,keepdim=True)[0].sum())/(ld[x].sum()+ld[y].sum())
    return ll
def cfm(ll,ld,dd,md,lm,mm):
    r1=torch.cat([ll,ld,lm],dim=1)
    r2=torch.cat([ld.T,dd,md.T],dim=1)
    r3=torch.cat([lm.T,md,mm],dim=1)
    fea=torch.cat([r1,r2,r3],dim=0)
    deg=torch.diag((torch.sum(fea>0,dim=1))**(-1/2)).double()
    G1=deg@fea@deg
    return fea,G1
class MyDataset(Dataset):
    def __init__(self,tri,ld):
        self.tri=tri
        self.ld=ld
    def __getitem__(self,idx):
        x,y=self.tri[idx,:]
        label=self.ld[x][y]
        return x,y,label
    def __len__(self):
        return self.tri.shape[0]
batch=32
ld,dd,md,lm,ll=load_data()
mm=calculate_sim(md,dd)
fea,G1=cfm(ll,ld,dd,md,lm,mm)
ti=torch.argwhere(ld>-1)
trset=DataLoader(MyDataset(ti,ld),batch,shuffle=True)
teset=DataLoader(MyDataset(ti,ld),batch,shuffle=False)
train_set,test_set=[],[]
for x1,x2,y in trset:
    train_set.append((x1,x2,y))
for x1,x2,y in teset:
    test_set.append((x1,x2,y))
torch.save(train_set,'./data/compare/hyper/50xls/trainset.pth')
torch.save(test_set,'./data/compare/hyper/50xls/testset.pth')
torch.save([ti,G1,fea],'./data/compare/hyper/50xls/par.pth')

In [ ]:
def train(model,train_set,test_set,fea,G1,tei,epoch,learn_rate):
    optimizer=torch.optim.Adam(model.parameters(),learn_rate,weight_decay=0.001)
    cost=nn.CrossEntropyLoss()
    model.train()
    fea,G1=fea.float().to(device),G1.float().to(device)
    Amax=[0,0]
    for i in range(epoch):
        for x1,x2,y in train_set:
            x1,x2,y=Variable(x1.long()).to(device),Variable(x2.long()).to(device),Variable(y.long()).to(device)
            out=model(x1,x2,fea,G1)
            loss=cost(out,y)+model.loss()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if (i+1)%1 == 0: #and i+1>=50:
            print(i)
            tacc(model,test_set,fea,G1,tei,Amax)
        #if i+1==epoch:
            #tacc(model,test_set,fea,G1,0,cros)
        torch.cuda.empty_cache()
def calculate_TPR_FPR(RD, f, B):
    old_id = np.argsort(-RD)
    min_f = int(min(f))
    max_f = int(max(f))
    TP_FN = np.zeros((RD.shape[0], 1), dtype=np.float64)
    FP_TN = np.zeros((RD.shape[0], 1), dtype=np.float64)
    TP = np.zeros((RD.shape[0], max_f), dtype=np.float64)
    TP2 = np.zeros((RD.shape[0], min_f), dtype=np.float64)
    FP = np.zeros((RD.shape[0], max_f), dtype=np.float64)
    FP2 = np.zeros((RD.shape[0], min_f), dtype=np.float64)
    P = np.zeros((RD.shape[0], max_f), dtype=np.float64)
    P2 = np.zeros((RD.shape[0], min_f), dtype=np.float64)
    for i in range(RD.shape[0]):
        TP_FN[i] = sum(B[i] == 1)
        FP_TN[i] = sum(B[i] == 0)
    for i in range(RD.shape[0]):
        for j in range(int(f[i])):
            if j == 0:
                if B[i][old_id[i][j]] == 1:
                    FP[i][j] = 0
                    TP[i][j] = 1
                    P[i][j] = TP[i][j] / (j + 1)
                else:
                    TP[i][j] = 0
                    FP[i][j] = 1
                    P[i][j] = TP[i][j] / (j + 1)
            else:
                if B[i][old_id[i][j]] == 1:
                    FP[i][j] = FP[i][j - 1]
                    TP[i][j] = TP[i][j - 1] + 1
                    P[i][j] = TP[i][j] / (j + 1)
                else:
                    TP[i][j] = TP[i][j - 1]
                    FP[i][j] = FP[i][j - 1] + 1
                    P[i][j] = TP[i][j] / (j + 1)
    ki = 0
    for i in range(RD.shape[0]):
        if TP_FN[i] == 0:
            TP[i] = 0
            FP[i] = 0
            ki = ki + 1
        else:
            TP[i] = TP[i] / TP_FN[i]
            FP[i] = FP[i] / FP_TN[i]
    for i in range(RD.shape[0]):
        kk = f[i] / min_f
        for j in range(min_f):
            TP2[i][j] = TP[i][int(np.round_(((j + 1) * kk))) - 1]
            FP2[i][j] = FP[i][int(np.round_(((j + 1) * kk))) - 1]
            P2[i][j] = P[i][int(np.round_(((j + 1) * kk))) - 1]
    TPR = TP2.sum(0) / (TP.shape[0] - ki)
    FPR = FP2.sum(0) / (FP.shape[0] - ki)
    Pr = P2.sum(0) / (P.shape[0] - ki)
    return TPR, FPR, Pr
def tacc(model,tset,fea,G1,tei,Amax):
    predall,yall=torch.tensor([]),torch.tensor([])
    model.eval()
    for x1,x2,y in tset:
        x1,x2,y=Variable(x1.long()).to(device),Variable(x2.long()).to(device),Variable(y.long()).to(device)
        pred=model(x1,x2,fea,G1).data
        predall=torch.cat([predall,torch.as_tensor(pred,device='cpu')],dim=0)
        yall=torch.cat([yall,torch.as_tensor(y,device='cpu')])
    pred=torch.softmax(predall,dim=1)[:,1]
    trh=torch.zeros(240,405)-1
    tlh=torch.zeros(240,405)-1
    trh[tei[:,0],tei[:,1]]=pred
    tlh[tei[:,0],tei[:,1]]=yall
    R=trh.numpy()
    label=tlh.numpy()
    f = np.zeros(shape=(R.shape[0], 1))
    for i in range(R.shape[0]):
        f[i] = np.sum(R[i] > -1)
    if min(f)>0:
        TPR,FPR,P=calculate_TPR_FPR(R,f,label)
        AUC=metrics.auc(FPR, TPR)
        AUPR=metrics.auc(TPR, P) + (TPR[0] * P[0])
        print("AUC:%.4f_AUPR:%.4f"%(AUC,AUPR))
        if AUPR>Amax[1]:
            Amax[0]=AUC
            Amax[1]=AUPR
            print("save")
            torch.save((predall,yall),"PandY")
ti,G1,fea=torch.load('./data/compare/hyper/50xls/par.pth')
train_set=torch.load('./data/compare/hyper/50xls/trainset.pth')
test_set=torch.load('./data/compare/hyper/50xls/testset.pth')
net=Net().to(device)
train(net,train_set,test_set,fea,G1,ti,epoch=15,learn_rate=0.0003)

In [ ]:
#n,ld,_,_,_,_,tei,_,_,_=torch.load('tempdata2.pth')
ti,_,_=torch.load('./data/compare/hyper/50xls/par.pth')
pred,_=torch.load('./PandY')
pred=torch.softmax(pred,dim=1)[:,1]
trh=torch.zeros(240,405)
trh[ti[:,0],ti[:,1]]=pred
lnc_dis_score=trh
index = np.argsort(-lnc_dis_score, axis=0) 
lncRNA_name=np.loadtxt("./data/yuguoxian_lncRNA_name.txt",dtype=str)
dis_name=pd.read_csv("./data/disease_name.txt",header=None,sep='\t')
lncRNA_all_50 = index[:30,:]
lncRNA_name_50 = lncRNA_all_50.T
candidate_list = []
for i in range(lncRNA_name_50.shape[0]): # 405
    for j in range(lncRNA_name_50.shape[1]): #50
        candidate_list.append([dis_name[2][i],lncRNA_name[lncRNA_name_50[i][j]],j+1,lnc_dis_score[lncRNA_name_50[i][j]][i]])
result = open('./data/compare/hyper/ST1.csv', 'w', encoding='gbk')
result.write('Disease Name,Candidate lncRNA name,Rank,Association score\n')
for m in range(len(candidate_list)): # 遍历的是405 * 50 的长度
    for n in range(len(candidate_list[m])): # 将每一行的元素求一下长度
        result.write(str(candidate_list[m][n])) # 一个一个的写入
        result.write(',') # 以 \t结束一行的写入
    result.write('\n') # 换行重新写
result.close() # 写完关闭文件

# Test